In [1]:
import os
from crewai import Crew, Agent, Task
from dotenv import load_dotenv

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI


load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"


# llm = ChatOpenAI(
#     temperature=0.7,
#     model_name="gpt-4o"
# )

In [ ]:
from crewai import Crew, Agent, Task

# verbos=True to print the conversation(AI Agent가 어떤식으로 사고하는지 알려준다.)

international_chef = Agent(
    role="International Chef",
    goal="Create ethnic cuisine recipies that are easy to cook at home",
    backstory="""
    You are an famous chef that specializes in couisine from countries all around the world.
    You know how to cook the most traditional dishes from all cultures but you also know how to adapt them for people to be able to cook them at home.
    """,
    verbose=True,
    allow_delegation=False,
    tools=[],
)

healthy_chef = Agent(
    role="Healthy Chef",
    goal="Turn any recipe into a healthy vegetarian recipe that is easy to cook with home ingredients.",
    backstory="""
    You are a chef specialized in healthy cooking.
    You can take any recipe and change the ingredients to make it vegetarian friendly without loosing the escense of the dish and what makes it delicious.
    """,
    verbose=True,
    allow_delegation=False,
    tools=[],
)

In [6]:
normal_recipe = Task(
    description="Come up with a {dish} that serves {people} people.",
    agent=international_chef,
    expected_output="Your answer MUST have three sections, the ingredients required with their quantities, the preparation instructions and serving suggestions",
    output_file="normal_recipe.md",
)

healthy_recipe = Task(
    description="Replace the ingredients of a recipe to make it vegetarian without making it less delicious, adjust if needed.",
    agent=healthy_chef,
    expected_output="Your answer MUST have four sections, the ingredients required with their quantities, the preparation instructions, serving suggestions and an explanation of the replaced ingredients.",
    output_file="healthy_recipe.md",
)

In [7]:

# task 는 순서대로 실행되기떄문에, 순서대로 task 를 추가해야한다.
# vervose=2 는 agent가 어떻게 작업을 하는지 알려준다.
crew = Crew(
    tasks=[
        normal_recipe,
        healthy_recipe,
    ],
    agents=[
        international_chef,
        healthy_chef,
    ],
    verbose=2,
)

result = crew.kickoff(
    inputs=dict(
        dish="Greek dinner",
        people="5",
    )
)

2025-01-14 20:34:49,958 - 8501035584 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: International Chef
 [INFO]: == Starting Task: Come up with a Greek dinner that serves 5 people.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 
To create a delightful Greek dinner that serves 5 people, I will provide a complete meal plan featuring some traditional Greek dishes that are easy to prepare at home. The meal will include a starter, a main course, and a dessert. Let's dive into the details:

**Starter: Greek Salad**

*Ingredients Required:*
- 3 large tomatoes, chopped
- 1 cucumber, sliced
- 1 green bell pepper, sliced
- 1 red onion, thinly sliced
- 150g feta cheese, cubed
- 10-15 kalamata olives
- 3 tablespoons extra virgin olive oil
- 1 tablespoon red wine vinegar
- 1 teaspoon dried oregano
- Salt and pepper to taste

*Preparation Instructions:*
1. In a large bowl, combine the tomatoes, cucumber, bell pepper, and red onion.
2. Add the feta cheese and olives on top of the vegetables.
3. Drizzle the olive o

# 16.5 Content Farm Crew (12:56)


In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"

In [25]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool


search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    tools=[
        search_tool,
        scrape_tool,
    ],
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read, informative, and makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
    tools=[
      search_tool,
      scrape_tool
    ],
    max_iter=10,
    llm=ChatOpenAI(model_name="gpt-4", temperature=0.7),
)

task = Task(
    description="""
      Write a blog post analyzing the {topic}.
      The post should include an introduction,
      at least three sub-sections examining the factors that led to the films' failures,
      links to relevant news articles or box office data,
      and a set of suggested hashtags for social media promotion.
    """,
    agent=editor,
    expected_output="""
      A well-structured blog post with a clear introduction,
      in-depth analysis of 3-5 major box office flops from 2024,
      supporting evidence from credible sources,
      and relevant hashtag suggestions for social media.
    """,
    output_file="blog_post.md",
)


crew = Crew(
    agents=[
        researcher,
        editor,
    ],
    tasks=[
      task
    ],
    verbose=2,
)


try:
    result = crew.kickoff(
        inputs=dict(
            topic="The biggest box office flops of 2024",
        ),
    )
except Exception as e:
    print(f"Error generating blog post: {e}")
    print("Trying again with modified prompt...")

    task.description = "Write a blog post analyzing the reasons behind the biggest box office failures of 2024. Provide in-depth insights and supporting data."
    result = crew.kickoff(
        inputs=dict(
            topic="The biggest box office flops of 2024",
        ),
    )


2025-01-15 08:07:32,313 - 8501035584 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: Senior Writer/Editor
 [INFO]: == Starting Task: Write a blog post analyzing the The biggest box office flops of 2024. The post should include an introduction, at least three sub-sections examining the factors that led to the films' failures, links to relevant news articles or box office data, and a set of suggested hashtags for social media promotion.


> Entering new CrewAgentExecutor chain...
I need to research about the biggest box office flops of 2024. I should find the movies that didn't perform well in the box office and analyze the reasons for their failure. I also need to search for relevant news articles or box office data to support my analysis. As for the hashtags for social media promotion, I should think about the main themes of the blog post and the movies discussed. It would be efficient to delegate the task of finding the biggest box office flops of 2024 and their box office data to the Senior Researcher, as this requires extensive and detail